In [1]:
import sys
import time

sys.path.append('../')


In [2]:
from filemanager.FolderManager import FolderManager
from svn_manager.svn_manager import get_svn_logs
import svn_manager.svn_data_factory as svnFactory

project_path = r'D:\dev\AutoPlanning\Pano\pano-task'

In [3]:
st = time.time()
source = get_svn_logs(project_path)
st = time.time()

(3058 / 3059) 82	3059) done. 196990 change file times 	


In [7]:
all_logs = []
all_diffs = []
for rv, (log, diffs) in  source.items():
    print(f'{rv} :{log.author}  {len(diffs)}')
    all_logs.append(log)
    all_diffs += diffs

7579 :jsj3  1
7578 :jsj3  1
7577 :jsj3  22
7576 :jsj3  2
7572 :jsj3  8
7571 :jsj3  1
7570 :jsj3  22
7567 :jsj3  1
7565 :jsj3  191
7564 :jsj3  1
7562 :jsj3  1
7559 :jsj3  2
7558 :kimjs  4
7557 :jsj3  2
7555 :jsj3  2
7553 :jsj3  1393
7517 :jsj3  17
7513 :jsj3  4
7510 :jsj3  46
7509 :jsj3  2
7508 :jsj3  5
7506 :jsj3  3
7504 :jsj3  6
7502 :jsj3  162
7500 :jsj3  4
7498 :jsj3  5
7497 :jsj3  34
7492 :jsj3  8
7490 :jsj3  5
7487 :jsj3  1823
7486 :jsj3  48
7485 :jsj3  446
7475 :parkjy  1
7474 :jsj3  1621
7473 :jsj3  3
7472 :jsj3  8
7464 :jsj3  5
7463 :jsj3  2
7461 :jsj3  2
7460 :jsj3  14
7459 :jsj3  187
7458 :jsj3  156
7377 :jsj3  7
7375 :jsj3  86
7365 :jsj3  11
7364 :jsj3  34
7360 :jsj3  19
7359 :jsj3  10
7342 :jsj3  1
7340 :jsj3  3
7339 :jsj3  141
7338 :jsj3  13
7333 :jsj3  81
7331 :jsj3  86
7276 :jsj3  11
7275 :jsj3  2
7274 :jsj3  11
7272 :jsj3  4
7266 :jsj3  52
7247 :jsj3  12
7245 :jsj3  3
7241 :jsj3  3
7240 :jsj3  1
7238 :jsj3  15
7235 :jsj3  33
7229 :jsj3  653
6553 :jsj3  8
6520 :jsj3  79


In [9]:
print(len(all_logs), " log")
print(len(all_diffs), " diffs")


3059  log
196990  diffs


In [10]:
from neo4j_manager.neo4jHandler import Neo4jHandler

In [15]:
neoDB = Neo4jHandler(uri="bolt://localhost:7687", user = "neo4j", database='svntest' , password= "123456789")
neoDB.print_info()
# neoDB.delete_all_nodes()

In [16]:
for log, diff in logs.values():
    print(log)
    print('====')
    print(diff)
    print(type(diff))
    break

Log(revision='7579', author='jsj3', date=datetime.datetime(2024, 11, 20, 2, 25, 9, 715007), msg='-로그인 처리 활성화')
====
[FileDiff(rv_path='7579#https://192.168.98.1/svn/automerge/branches/AutoPlanning_Pano\\AppCommon_AP\\ToolbarAPWorkflow.cpp', revision='7579', filepath='D:\\dev\\AutoPlanning\\Pano\\pano-task\\AppCommon_AP\\ToolbarAPWorkflow.cpp', repo_path='https://192.168.98.1/svn/automerge/branches/AutoPlanning_Pano\\AppCommon_AP\\ToolbarAPWorkflow.cpp', action=<DiffActionType.Mod: 'Modified'>)]
<class 'list'>


In [21]:
def make_batch(data_list ,batch_size = 1000):
    batch_list = []
    for i in range(0, len(data_list), batch_size):
        batch = data_list[i:i+batch_size]
        batch_list.append(batch)
    
    print(len(batch_list))
    for_check = set()
    for batch in batch_list:
        for_check.add(len(batch))
    
    # print('batch_list len : ', len(batch_list))
    return batch_list

#데이터 생산 검증
batch_list = make_batch(all_diffs)
for_check = set()
for batch in batch_list:
    for_check.add(len(batch))

print(f"length {len(batch_list)}, {for_check}")


197
batch_list len :  197
length 197, {1000, 990}


In [22]:
import time
st = time.time()
print("start Log")
batch_list = make_batch(all_logs)
for idx, batch in enumerate(batch_list):
    neoDB.save_data(batch, pid_key='revision')
    print(f'\r{idx}/{len(batch_list)} \t', end="")
print(f'\r{len(batch_list)} done\t', end="")
ed = time.time()

print('save log db time ', ed-st)


st = time.time()
print("start FileDiff")
batch_list = make_batch(all_diffs)
for idx, batch in enumerate(batch_list):
    neoDB.save_data(batch, pid_key='rv_path')
    print(f'\r{idx}/{len(batch_list)} \t', end="")
print(f'\r{len(batch_list)} done\t', end="")
ed = time.time()

print('save FileDiff db time ', ed-st)


start Log
4
batch_list len :  4
4 done	save log db time  7.729941129684448
start FileDiff
197
batch_list len :  197
197 done	save FileDiff db time  8670.59530377388
